In [3]:
import pandas as pd
import numpy as np
from ipywidgets import interact
from ipywidgets import IntSlider
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
%matplotlib inline

/home/tronco/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Autoregressive Integrated Moving Average (ARIMA) Model

In [17]:
# Loading data
b08 = pd.read_csv('data/b08c2.csv')
b08_viento = b08["viento"]

train_per = 0.6
data_len = b08_viento.shape[0]
data = b08_viento[:int(data_len*train_per)].as_matrix()
test = b08_viento[int(data_len*train_per):].as_matrix()

In [19]:
p=2; q=0; r =2
params = (p,q,r)
forecast = []
w_speed = []
for i,value in enumerate(test):
    data = np.append(data,value)
    model = ARIMA(data,params)
    result = model.fit()
    pred = result.forecast(steps=12)
    w_speed.append(test[i:i+12])
    forecast.append(pred[0])


/home/tronco/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


KeyboardInterrupt: 

In [18]:
test

array([ 8.4,  9.7,  8.5, ...,  3.8,  1. ,  1.7])

In [20]:
forecast = np.array(forecast)
w_speed = np.array(w_speed)

In [23]:
def plot_forecast(serie,forecast,step=1):
    i = int(step-1)
    fig = plt.figure(figsize=(20,5))
    ax = fig.add_subplot(111)
    ax.plot(forecast[:100,i],'r')
    ax.plot(serie[i:100+i])
    ax.legend(["Forecast", "Wind Speed"])
    plt.show()
    

def interact_plot(step = 1):
    plot_forecast(test,forecast,step)
    
interact(interact_plot,step=(1,12))

<function __main__.interact_plot>

In [37]:
np.linalg.norm(forecast-w_speed,axis=0)/forecast.shape[0]

array([ 0.01761584,  0.05327291,  0.06638938,  0.07123832,  0.07216189,
        0.07205067,  0.07197215,  0.0719194 ,  0.07198955,  0.0721584 ,
        0.07240565,  0.07248441])